# Imports

In [1]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch_optimizer as optim
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
import torchvision
warnings.filterwarnings('ignore')

from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")
set_global("dataloader_workers", 4)
set_global("use_autocast", True)
set_global("models_dir", "/home/ahemf/cache/")

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook, set_device, my_collate
get_global("cache_dir")
from facebook_hateful_memes_detector.models import Fasttext1DCNNModel, MultiImageMultiTextAttentionEarlyFusionModel, LangFeaturesModel, AlbertClassifer
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut, ImageAugment
from facebook_hateful_memes_detector.training import *
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)
device


device(type='cuda')

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import MarianMTModel, MarianTokenizer
from tqdm.auto import tqdm, trange


In [2]:
data = get_datasets(data_dir="../data/", train_text_transform=None, train_image_transform=None, 
                    test_text_transform=None, test_image_transform=None, 
                    cache_images = True, use_images = False, dev=False, test_dev=True,
                    keep_original_text=False, keep_original_image=False, 
                    keep_processed_image=True, keep_torchvision_image=False,)

In [3]:
data["train"]

,id,img,label,text
0,42953,../data/img/42953.png,0,its their character not their color that matters
1,23058,../data/img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,../data/img/13894.png,0,putting bows on your pet
3,37408,../data/img/37408.png,0,i love everything and everybody! except for sq...
4,82403,../data/img/82403.png,0,"everybody loves chocolate chip cookies, even h..."
...,...,...,...,...
8495,10423,../data/img/10423.png,1,nobody wants to hang auschwitz me
8496,98203,../data/img/98203.png,1,when god grants you a child after 20 years of ...
8497,36947,../data/img/36947.png,1,gays on social media: equality! body positivit...
8498,16492,../data/img/16492.png,1,having a bad day? you could be a siamese twin ...


# DL/Transformers based Aug

In [5]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
text = 'have you ever studied the history of the jews? did you know that they have always banded together as a tribe, infiltrated governments.'

text_long = 'have you ever studied the history of the jews? did you know that they have always banded together as a tribe, infiltrated governments, monopolized the financial systems of nations instigated wars and intentionally created chaos in societies? the jews have mass murdered millions of non- jews over the centuries they have seized control of the media so you will never find out study the history of the jews!'


- summary
    - with synonym replacement
    - with word2vec based replacement
    - with word cutout and stopword insert
    - word split and word join
- qna with various questions
    - with synonym replacement
    - with word2vec based replacement
    - with word cutout and stopword insert
    - word split and word join
    - QnA over summary
    - QnA over translation
- Back translate
    - with synonym replacement
    - with word2vec based replacement
    - with word cutout and stopword insert
    - word split and word join
    - Over summary


## QnA

In [6]:


# twmkn9/albert-base-v2-squad2 , twmkn9/distilroberta-base-squad2, huggingface/prunebert-base-uncased-6-finepruned-w-distil-squad
# mrm8488/bert-tiny-finetuned-squadv2 , mrm8488/bert-small-finetuned-squadv2, mrm8488/bert-mini-finetuned-squadv2


bert_qna_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
bert_qna_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

albert_qna_tokenizer = AutoTokenizer.from_pretrained("twmkn9/albert-base-v2-squad2")
albert_qna_model = AutoModelForQuestionAnswering.from_pretrained("twmkn9/albert-base-v2-squad2")

tiny_bert_qna_tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")
tiny_bert_qna_model = AutoModelForQuestionAnswering.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")


distilbert_qna_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
distilbert_qna_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

from transformers import XLNetTokenizer, XLNetForQuestionAnswering, XLNetForQuestionAnsweringSimple
import torch

XLNet_qna_tokenizer =  XLNetTokenizer.from_pretrained('xlnet-base-cased')
XLNet_qna_model = XLNetForQuestionAnsweringSimple.from_pretrained('xlnet-base-cased')


qna_models = dict(bert=dict(tokenizer=bert_qna_tokenizer, model=bert_qna_model),
                 albert=dict(tokenizer=albert_qna_tokenizer, model=albert_qna_model),
                 tiny_bert=dict(tokenizer=tiny_bert_qna_tokenizer, model=tiny_bert_qna_model),
                 distilbert=dict(tokenizer=distilbert_qna_tokenizer, model=distilbert_qna_model),
                 XLNet=dict(tokenizer=XLNet_qna_tokenizer, model=XLNet_qna_model))





In [9]:

def get_question_answering(initial_aug=None, qna_model: dict=qna_models["bert"]):
    questions = ["Is this offensive?", 
                 "What part is offensive, bad, misinformed, hurtful?", 
                 "Blatantly misguiding and forming opinion?", 
                 "Only opinions are expressed?",
                "Targeted towards a particular race, gender, community?", 
                 "Is this hate speech?",
                "Is there a show of disdain and cynicism?",
                "Who is responsible and who should be blamed?",
                "Are they doing the right thing? Should they be corrected?",
                "We need to stop them!",
                "Us vs them,",
                 "What is happening?","Why?", "How", "When did they?", "Thier ways?"]
    from collections import defaultdict
    def transform(text):
        tokenizer = qna_model["tokenizer"]
        model = qna_model["model"]
        with torch.no_grad():
            text = initial_aug(text) if initial_aug is not None else text
            answers = []
            batch_inputs = defaultdict(list)
            for question in questions:
                inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt", pad_to_max_length=True, max_length=256)
                for k, v in inputs.items():
                    batch_inputs[k].append(v)

            for k, v in batch_inputs.items():
                batch_inputs[k] = torch.cat(v, 0)
            answer_start_scores, answer_end_scores = model(**batch_inputs)
            answer_start = torch.argmax(answer_start_scores, 1)  # Get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores, 1) + 1 
            for i, input_ids in  enumerate(batch_inputs["input_ids"]):
                answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start[i]:answer_end[i]]))
                answers.append(answer)
            answers = list(set([a for a in answers if len(a.split())>=2]))
            return answers
        
    return transform

qna = get_question_answering(qna_model=qna_models["distilbert"])


In [10]:
qna('have you ever studied the history of the jews? did you know that they have always banded together as a tribe, infiltrated governments.')


['they have always banded together as a tribe , infiltrated governments',
 'infiltrated governments']

## Summarizer

In [11]:
from transformers import AutoModelWithLMHead, AutoTokenizer

t5_summarizer_model = AutoModelWithLMHead.from_pretrained("t5-large")
t5_summarizer_tokenizer = AutoTokenizer.from_pretrained("t5-large")

bart_summarizer_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_summarizer_model = AutoModelWithLMHead.from_pretrained("facebook/bart-large-cnn")

ctrl_summarizer_tokenizer = AutoTokenizer.from_pretrained("ctrl")
ctrl_summarizer_model = AutoModelWithLMHead.from_pretrained("ctrl")



summary_models = dict(ctrl=dict(tokenizer=ctrl_summarizer_tokenizer, model=ctrl_summarizer_model),
                    bart=dict(tokenizer=bart_summarizer_tokenizer, model=bart_summarizer_model),
                    t5=dict(tokenizer=t5_summarizer_tokenizer, model=t5_summarizer_model))





In [13]:

def get_summarizer(models, initial_aug=None, ):
    from collections import defaultdict
    def transform(text):
        summaries = []
        with torch.no_grad():
            for k, m in models.items():
                tokenizer = m["tokenizer"]
                model = m["model"]
                _ = model.eval()
                inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=128)
                outputs = model.generate(inputs, max_length=32, min_length=16, length_penalty=2.0, num_beams=4, early_stopping=True)
                summ = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0]))
                summaries.append(summ)
            return summaries
    return transform

summary_generator = get_summarizer(summary_models)
summary_generator(ARTICLE)
summary_generator(text)


['summarize: New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.\n A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.\n Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.\n In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.\n Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in',
 '</s><s>Liana Barrientos has been married five times since she was 23 years old. She got married in Westchester County, New York, in',
 'Liana barrientos declared "i do" five more times, sometimes within two weeks of each other . in 2010, she married once more,']

['summarize: have you ever studied the history of the jews? did you know that they have always banded together as a tribe, infiltrated governments.',
 '</s><s>The jews have always banded together as a tribe, infiltrated governments. Have you ever studied the history of the jews? did you know',
 'jews have always banded together as a tribe, infiltrated governments . they have always banded together as a']

##  Translation

In [9]:
btdict = dict()

# model_name = 'Helsinki-NLP/opus-mt-en-tn'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# btdict["en-tn"] = dict(fwd=(tokenizer, model))

# model_name = 'Helsinki-NLP/opus-mt-tn-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

# btdict["en-tn"]["inv"] = (tokenizer, model)


model_name = 'Helsinki-NLP/opus-mt-en-ru'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
btdict["en-ru"] = dict(fwd=(tokenizer, model))

model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

btdict["en-ru"]["inv"] = (tokenizer, model)


# model_name = 'Helsinki-NLP/opus-mt-en-de'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# btdict["en-de"] = dict(fwd=(tokenizer, model))

# model_name = 'Helsinki-NLP/opus-mt-de-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

# btdict["en-de"]["inv"] = (tokenizer, model)


# model_name = 'Helsinki-NLP/opus-mt-en-CELTIC'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# btdict["en-CELTIC"] = dict(fwd=(tokenizer, model))

# model_name = 'sshleifer/opus-mt-CELTIC-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

# btdict["en-CELTIC"]["inv"] = (tokenizer, model)


# model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# btdict["en-ROMANCE"] = dict(fwd=(tokenizer, model))

# model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

# btdict["en-ROMANCE"]["inv"] = (tokenizer, model)

# model_name = 'Helsinki-NLP/opus-mt-en-chk'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# btdict["en-chk"] = dict(fwd=(tokenizer, model))

# model_name = 'Helsinki-NLP/opus-mt-chk-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

# btdict["en-chk"]["inv"] = (tokenizer, model)

# model_name = 'Helsinki-NLP/opus-mt-en-jap'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)
# btdict["en-jap"] = dict(fwd=(tokenizer, model))

# model_name = 'Helsinki-NLP/opus-mt-jap-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name)

# btdict["en-jap"]["inv"] = (tokenizer, model)




In [16]:
from transformers import MarianMTModel, MarianTokenizer

def get_back_translation_model(btdict):
    def back_translate(text):
        texts = [text]
        tl = len(text.split())
        answers = []
        for k, v in btdict.items():
            fwd_tokenizer, fwd_model = v["fwd"]
            inv_tokenizer, inv_model = v["inv"]
            lang_codes = fwd_tokenizer.supported_language_codes
            if "ROMANCE" in k:
                lang_codes = ['>>fr<<', '>>es<<', '>>it<<', '>>pt<<', '>>ro<<', '>>ca<<', '>>gl<<', '>>la<<', '>>wa<<', '>>fur<<', '>>oc<<', '>>sc<<', '>>an<<', '>>frp<<',]
            if len(lang_codes) > 0:
                texts = [t for text in texts for t in [lang+" "+text for lang in lang_codes]]
            translated = fwd_model.generate(**fwd_tokenizer.prepare_translation_batch(texts))
            fwd_translations = [fwd_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
            inv_batch = inv_tokenizer.prepare_translation_batch(fwd_translations)
            translated = inv_model.generate(**inv_batch)
            tgt_text = [inv_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
            answers.append(tgt_text)
        answers = [a for ans in answers for a in ans]
        ans_lens = [len(a.split()) for a in answers]
        answers = list(set([a for l,a in zip(ans_lens,answers) if l>=2 and l>=tl/2]))
        return answers
    return back_translate

    

In [17]:
back_translate = get_back_translation_model(btdict)
back_translate("The cat sat on the front porch sipping a pint of milk.")


['Кошка сидела на крыльце, потягивая пинту молока.']
{'input_ids': tensor([[ 5468,  7555, 18523,    41,    17,    42,  3170,  7795,    30,     2,
            15,   275,  8020,  2434,   106,    21, 23045,  2055, 36515,     3,
             0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[62517,    32, 12936, 19046,    25,     4,   562,   649,  2161,     2,
         29932,    13,   562,  7307,     5, 18675,     3]])


['The cat sat on the porch, pulling a pint of milk.']

In [71]:
%timeit back_translate(text)

43.8 s ± 3.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
models_dict= dict()
import torch
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')
en2de_cp4 = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de',
                       checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
                       tokenizer='moses', bpe='fastbpe')

de2en_cp4 = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en',
                       checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
                       tokenizer='moses', bpe='fastbpe')
en2de_wmt16 = torch.hub.load('pytorch/fairseq', 'transformer.wmt16.en-de', tokenizer='moses', bpe='fastbpe')
en2de_wmt17 = torch.hub.load('pytorch/fairseq', 'conv.wmt17.en-de', tokenizer='moses', bpe='fastbpe')
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

models_dict["en-de"] = dict(fwd=[en2de, en2de_cp4, en2de_wmt16, en2de_wmt17], inv=[de2en, de2en_cp4])
models_dict["en-ru"] = dict(fwd=[en2ru], inv=[ru2en])


        
    
    

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /home/ahemf/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp
cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating /local/home/ahemf/.cache/torch/hub/pytorch_fairseq_master/build
creating /local/home/ahemf/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7
creating /local/home/ahemf/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/fairseq
creating /local/home/ahemf/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/fairseq/clib
creating /local/home/ahemf/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
Emitting ninja build file /local/home/ahemf/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
creating build/l

100%|██████████| 2988854245/2988854245 [00:59<00:00, 50220417.19B/s]
Using cache found in /home/ahemf/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 2992273886/2992273886 [01:01<00:00, 48385103.15B/s]
Using cache found in /home/ahemf/.cache/torch/hub/pytorch_fairseq_master
 44%|████▍     | 5265638400/11946275315 [02:02<03:36, 30829950.00B/s] 

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
def get_pytorch_backtranslate(mdict):
    def py_translate(text):
        tl = len(text.split())
        answers = []
        for k, v in mdict.items():
            fwd = v["fwd"]
            inv = v["inv"]
            for fm in fwd:
                intermediate = fm.translate(text)
                for bm in inv:
                    res = bm.translate(intermediate)
                    answers.append(res)
        ans_lens = [len(a.split()) for a in answers]
        answers = list(set([a for l,a in zip(ans_lens,answers) if l>=2 and l>=tl/2]))
        return answers
    return py_translate

py_translate = get_pytorch_backtranslate(models_dict)
py_translate("The cat sat on the front porch sipping a pint of milk.")


 44%|████▍     | 5268157440/11946275315 [02:20<03:36, 30829950.00B/s]

## Text Generation [x]

In [20]:
gen_model_names = ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", "microsoft/DialoGPT-large"]

gen_models = [pipeline("text-generation")] + [pipeline("text-generation", model=m, tokenizer=m) for m in gen_model_names]


In [34]:
def get_gen_models(gen_models):
    def generate(text, nlp):
        spaced_tokens = text.split()
        n_tokens = len(spaced_tokens)
        nlp1 = lambda t: nlp(t,max_length=n_tokens*2, clean_up_tokenization_spaces=True, do_sample=True, top_p=0.97, top_k=500)[0]['generated_text']
        nlp2 = lambda t: nlp(t,max_length=n_tokens*2, clean_up_tokenization_spaces=True, do_sample=True, top_p=0.95, top_k=100)[0]['generated_text']
        if n_tokens <= 3:
            return [text]
        t1 = nlp1(text)
        t2 = nlp1(" ".join(spaced_tokens[:int(n_tokens/2)]))
        t3 = nlp1(" ".join(spaced_tokens[int(n_tokens/2):]))
        t4 = nlp2(text)
        t5 = nlp2(" ".join(spaced_tokens[:int(n_tokens/2)]))
        t6 = nlp2(" ".join(spaced_tokens[int(n_tokens/2):]))
        
        t7 = nlp1(text + ". Our opinion is")
        t8 = nlp1(text + ". You know")
        t9 = nlp1(text + ". They are well known for")
        t10 =  nlp1(text + ". I think they")
        t11 = nlp1(text + ". Their poeple have done")
        t12 = nlp1(text + ". In the past those guys")
        
        return [t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12]
    
    def transform(text):
        answers=[]
        for i, nlp in enumerate(gen_models):
            generated = generate(text, nlp)
            generated = [str(i) + ": "+ g for g in generated]
            generated[-1] = generated[-1] + "\n" + "-"*100 + "\n"
            answers.extend(generated)
        return answers
    return transform

text_gen = get_gen_models(gen_models)
text_gen(text)


2020-06-07 16:17:48 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:17:51 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:17:55 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:17:57 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:18:02 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:18:05 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:18:07 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:18:10 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:18:13 WARNING: Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
2020-06-07 16:18:15 WARNING: Setting `pad_token_id` to 

['0: have you ever studied the history of the jews? did you know that they have always banded together as a tribe, infiltrated governments. Did you ever know that jews know the unspoken regulations of the land that our country',
 "0: have you ever studied the history of the jews? did you all somehow think the jews were good, if not the fair? did you know there were hundreds of 'ducks' of coloured parts, only four-fif",
 "0: know that they have always banded together as a tribe, infiltrated governments. But by now, it's all too familiar. At the very least, they have known each other before.\n\nThis summer, more and more people",
 '0: have you ever studied the history of the jews? did you know that they have always banded together as a tribe, infiltrated governments. now they have joined together as a community?"\n\n\nAnd, "but I know all',
 "0: have you ever studied the history of the jews? did you see a lot of them?\n\n\nLurker: Never. People did know about these people. They don't tal

## Fill mask

In [32]:
fill_mask_model_names = [
    "distilroberta-base", "bert-large-uncased-whole-word-masking",
    "albert-base-v2", "Hate-speech-CNERG/dehatebert-mono-english",
    "novinsh/xlm-roberta-large-toxicomments-12k",
    "allenai/reviews_roberta_base", "ssun32/bert_twitter_turkle",
    "activebus/BERT_Review", "sap-ai-research/BERT-Large-Contrastive-Self-Supervised-ACL2020"
]

# "mrm8488/t5-base-finetuned-span-sentiment-extraction",
# "mrm8488/distilroberta-base-finetuned-sentiment",

fill_mask_models = [pipeline("fill-mask")] + [
    pipeline("fill-mask", model=m, tokenizer=m) for m in fill_mask_model_names
]

In [26]:
import random
def get_fill_mask(fill_mask_models):
    def fill_mask_ntimes(text, nlp, n=2):
        mask = nlp.tokenizer.mask_token
        for _ in range(n):
            spaced_tokens = clean_text(text).split()
            n_tokens = len(spaced_tokens)
            token_lengths = list(map(len, spaced_tokens))
            token_lengths = np.array([0 if t<=2 else t for t in token_lengths])
            token_probas = token_lengths / token_lengths.sum()
            mask_pos = np.random.choice(list(range(n_tokens)), 1, replace=False, p=token_probas)[0]
            spaced_tokens[mask_pos] = mask
            masked_text = " ".join(spaced_tokens)
            replies = nlp(masked_text)
            text = [r['sequence'] for r in replies][0]
        return text
            
        
    
    def fill_mask(text):
        spaced_tokens = clean_text(text).split()
        n_tokens = len(spaced_tokens)
        token_lengths = list(map(len, spaced_tokens))
        token_lengths = np.array([0 if t<=2 else t for t in token_lengths])
        token_probas = token_lengths / token_lengths.sum()
        answers = []
        for i, nlp in enumerate(fill_mask_models):
            mask = nlp.tokenizer.mask_token
            mask_pos = np.random.choice(list(range(n_tokens)), 3, replace=False, p=token_probas)
            for p in mask_pos:
                tks = list(spaced_tokens)
                tks[p] = mask
                masked_text = " ".join(tks)
                replies = nlp(masked_text)
                replies = [r['sequence'] for r in replies][:2]
                answers.extend(replies)
        answers = list(set([clean_text(a) for a in answers]))
        return answers
    
    def fill_mask(text):
        answers = []
        for i, nlp in enumerate(fill_mask_models):
            t = fill_mask_ntimes(text, nlp, n=2)
            answers.append(t)
        answers = list(set([clean_text(a) for a in answers]))
        return answers
    
    return fill_mask

    

fill_mask = get_fill_mask(fill_mask_models)
fill_mask(text)

        
            
            

NameError: name 'fill_mask_models' is not defined